In [1]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [2]:
data = np.array([49,58,75,110,112,132,151,276,281,362])

In [3]:
m0 = 200
stats.wilcoxon(data - m0)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [4]:
a = np.array([22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15])
b= np.array([17,18,18,15,12,4,14,15,10])
stats.mannwhitneyu(a, b)

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

In [6]:
data = pd.read_csv('challenger.txt', sep = '\t', header = 0)

In [7]:
data.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [16]:
data_0 = data[(data['Incident'] == 0)].Temperature
data_1 = data[(data['Incident'] == 1)].Temperature

In [53]:
data_a = np.array(data_0)
data_a
data_b = np.array(data_1)
data_b

array([21.1, 13.9, 17.2, 21.1, 11.7, 23.9, 14.4])

In [20]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)
np.random.seed(0)
print "p-value: %f" % permutation_test(data_0, data_1, max_permutations = 10000)

p-value: 0.007000


In [66]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
n_samples = 1000
np.random.seed(0)
a_samples = get_bootstrap_samples(data_a, n_samples)
b_samples = get_bootstrap_samples(data_b, n_samples)
deltas = map(lambda x: x[0].mean() - x[1].mean(), zip(a_samples, b_samples))
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

intervals = stat_intervals(deltas, 0.05)

In [71]:
round(intervals[0], 4)

1.423